In [41]:
import openpyxl #biblioteca para abrir arquivos em excel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV # biblioteca para treino da arvore de decisão e otimização de hiperparâmetros
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor #para classificação e regressão as variáveis
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix # biblioteca para métricas do modelo
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier # biblioteca de rede neural

In [42]:
#comando para exibição de todas as colunas do meio do dataframe que, este este, seriam ocultadas pelo jupyter.
pd.set_option('display.max_columns', 25)

In [43]:
#carregando os dados
bd1 = pd.read_excel("defaultCreditcardClients.xls")

In [44]:
# Aqui retiramos a linha onde tinha apenas o nome da variável, pois esta estava sendo contada como valor sem o ser.
#Logo, bd1_copy é o mesmo bd1 só que sem os cabeçalhos descritivos
bd1_copy = bd1.iloc[1:]
bd1_copy

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
1,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,29996,220000,1,3,1,39,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29997,29997,150000,1,3,2,43,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29998,29998,30000,1,2,2,37,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1
29999,29999,80000,1,3,1,41,1,-1,0,0,0,-1,-1645,78379,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [45]:
#exclusão da coluna 'Unnamed: 0'
bd1.drop('Unnamed: 0', axis=1, inplace=True)

In [46]:
# Dividindo os dados em features eixo(X) e variável alvo eixo(y)
X = bd1_copy.drop(['Y'],  axis=1)
y = bd1_copy['Y'].astype(int)

In [47]:
# Normalizando os dados usando StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [48]:
# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [49]:
# Definindo modelos individuais
tree_model = RandomForestClassifier()
neural_network_model = MLPClassifier()

In [50]:
# Crie o ensemble com VotingClassifier
ensemble_model = VotingClassifier(estimators=[
    ('tree', tree_model),
    ('neural_network', neural_network_model)
], voting='hard')  # 'hard' para votação por maioria, 'soft' para votação ponderada


In [51]:
# Range de hiperparâmetros
param_dist = {
    'tree__n_estimators': [50, 100, 200],
    'tree__max_depth': [None, 10, 20],
    'tree__min_samples_split': [2, 5, 10],
    'neural_network__hidden_layer_sizes': [(50,), (100, 50), (100, 50, 20)],
    'neural_network__max_iter': [500, 1000],
    'neural_network__alpha': [0.0001, 0.001, 0.01]  # Parâmetro de regularização para a rede neural
}

In [52]:
# Verificação de melhores hiperpametros no random
random_search = RandomizedSearchCV(estimator=ensemble_model, param_distributions=param_dist, cv=5, n_iter=10, random_state=42)
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=VotingClassifier(estimators=[('tree',
                                                           RandomForestClassifier()),
                                                          ('neural_network',
                                                           MLPClassifier())]),
                   param_distributions={'neural_network__alpha': [0.0001, 0.001,
                                                                  0.01],
                                        'neural_network__hidden_layer_sizes': [(50,),
                                                                               (100,
                                                                                50),
                                                                               (100,
                                                                                50,
                                                                                20)],
                                        'neural_network__max_iter': [500, 1000],
                                        'tree__max_depth': [None, 10, 20],
                                        'tree__min_samples_split': [2, 5, 10],
                                        'tree__n_estimators': [50, 100, 200]},
                   random_state=42)

In [53]:
# comando para trazer os melhores hiperparâmetros
best_params = random_search.best_params_
print("Melhores parâmetros encontrados pela pesquisa aleatória:", best_params)

Melhores parâmetros encontrados pela pesquisa aleatória: {'tree__n_estimators': 50, 'tree__min_samples_split': 10, 'tree__max_depth': 10, 'neural_network__max_iter': 1000, 'neural_network__hidden_layer_sizes': (100, 50), 'neural_network__alpha': 0.01}


In [54]:
# Avalie o modelo ensemble com os melhores parâmetros no conjunto de teste
y_pred = random_search.best_estimator_.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      1.00      0.88      4673
           1       0.71      0.02      0.04      1327

    accuracy                           0.78      6000
   macro avg       0.74      0.51      0.46      6000
weighted avg       0.77      0.78      0.69      6000



In [55]:
print(confusion_matrix(y_test,y_pred))

[[4661   12]
 [1298   29]]
